# Silver - NYC Yellow Taxi Trip Data

## 0. Setup

In [1]:
import re
from collections.abc import Callable
from datetime import datetime
from datetime import time
from typing import Any

import findspark

from shared import PaymentTypeNYC
from shared import RateCodeNYC
from shared import VendorNYC

findspark.init()

In [2]:
from pyspark import RDD
from pyspark import StorageLevel
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import types as t

from utils import setup_spark

spark: SparkSession = setup_spark(title="Silver NYC")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/11/28 14:36:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
debug_exploration: bool = True
debug_dropped: bool = True
explore_storage_level: StorageLevel = StorageLevel.MEMORY_AND_DISK

In [4]:
from schemas import schema_nyc_bronze

df_nyc: DataFrame = spark.read.schema(schema_nyc_bronze).csv(
    path="data/nyc/yellow_tripdata_*.csv",
    header=True,
)

rdd_nyc: RDD[t.Row] = df_nyc.rdd

## 1. Prerequisites

In [5]:
def parse_datetime(x: str) -> datetime:
    return datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [6]:
import shared

REGEX_DATETIME = re.compile(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}")
REGEX_FLOAT = re.compile(r"[+-]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?")
REGEX_INTEGER = re.compile(r"[+-]?\d+")
VENDORS_VALID_VALUES: set[str] = {str(vendor.value) for vendor in list(VendorNYC)}
RATE_CODE_VALID_VALUES: set[str] = {str(rate_code.value) for rate_code in list(RateCodeNYC)}
PAYMENT_TYPE_VALID_VALUES: set[str] = {str(rate_code.value) for rate_code in list(PaymentTypeNYC)}
STORE_AND_FORWARD_VALID_VALUES: set[str] = {"Y", "N"}


def is_float_valid(x: str) -> bool:
    if x is None:
        return False
    return REGEX_FLOAT.fullmatch(x)


def is_int_valid(x: str) -> int:
    if x is None:
        return False
    return REGEX_INTEGER.fullmatch(x)


def is_coordinate_line_valid(x: str, range_min: float = -180, range_max: float = 180) -> bool:
    if not is_float_valid(x):
        return False
    if float(x) == 0:
        return True
    return range_min <= float(x) <= range_max


def is_vendor_valid(x: str) -> bool:
    return x in VENDORS_VALID_VALUES


def is_pickup_time_valid(x: str) -> bool:
    return REGEX_DATETIME.fullmatch(x) is not None


def is_dropoff_time_valid(x: str) -> bool:
    return REGEX_DATETIME.fullmatch(x) is not None


def is_passenger_count_valid(x: str) -> bool:
    if not is_int_valid(x):
        return False
    return shared.PASSENGER_MIN <= int(x) <= shared.PASSENGER_MAX


def is_trip_distance_valid(x: str) -> bool:
    if not is_float_valid(x):
        return False
    return shared.DISTANCE_MIN <= float(x) <= shared.DISTANCE_MAX


def is_pickup_longitude_valid(x: str) -> bool:
    return is_coordinate_line_valid(
        x,
        shared.LOCATION_LONGITUDE_NYC - shared.LOCATION_LINE_TOLERANCE,
        shared.LOCATION_LONGITUDE_NYC + shared.LOCATION_LINE_TOLERANCE,
    )


def is_pickup_latitude_valid(x: str) -> bool:
    return is_coordinate_line_valid(
        x,
        shared.LOCATION_LATITUDE_NYC - shared.LOCATION_LINE_TOLERANCE,
        shared.LOCATION_LATITUDE_NYC + shared.LOCATION_LINE_TOLERANCE,
    )


def is_rate_code_valid(x: str) -> bool:
    return x in RATE_CODE_VALID_VALUES


def is_store_and_forward_valid(x: str) -> bool:
    return x in STORE_AND_FORWARD_VALID_VALUES


def is_dropoff_longitude_valid(x: str) -> bool:
    return is_coordinate_line_valid(x, shared.LOCATION_LONGITUDE_NYC - 1, shared.LOCATION_LONGITUDE_NYC + 1)


def is_dropoff_latitude_valid(x: str) -> bool:
    return is_coordinate_line_valid(x, shared.LOCATION_LATITUDE_NYC - 1, shared.LOCATION_LATITUDE_NYC + 1)


def is_payment_type_valid(x: str) -> bool:
    return x in PAYMENT_TYPE_VALID_VALUES


def is_fare_amount_valid(x: str) -> bool:
    if not is_float_valid(x):
        return False
    return shared.FARE_MIN <= float(x) <= shared.FARE_MAX


def is_extra_valid(x: str) -> bool:
    if not is_float_valid(x):
        return False
    return shared.EXTRAS_MIN <= float(x) <= shared.EXTRAS_MAX


def is_mta_tax_valid(x: str) -> bool:
    if not is_float_valid(x):
        return False
    return shared.MTA_TAX_MIN <= float(x) <= shared.MTA_TAX_MAX


def is_tip_amount_valid(x: str) -> bool:
    if not is_float_valid(x):
        return False
    return shared.TIPS_MIN <= float(x) <= shared.TIPS_MAX


def is_tolls_amount_valid(x: str) -> bool:
    if not is_float_valid(x):
        return False
    return shared.TOLLS_MIN <= float(x) <= shared.TOLLS_MAX


def is_improvement_surcharge_valid(x: str) -> bool:
    if not is_float_valid(x):
        return False
    return shared.SURCHARGE_MIN <= float(x) <= shared.SURCHARGE_MAX


def is_total_amount_valid(x: str) -> bool:
    if not is_float_valid(x):
        return False
    return shared.TOTAL_MIN <= float(x) <= shared.TOTAL_MAX


def is_duration_valid(row: t.Row) -> bool:
    pickup_time = parse_datetime(row["tpep_pickup_datetime"])
    dropoff_time = parse_datetime(row["tpep_dropoff_datetime"])

    return shared.SECONDS_MIN <= (dropoff_time - pickup_time).total_seconds() <= shared.SECONDS_MAX


def is_total_amount_sum_valid(row: t.Row) -> bool:
    total_amount = float(row["total_amount"])
    total_amount_calculated = round(
        float(row["fare_amount"])
        + float(row["extra"])
        + float(row["mta_tax"])
        + float(row["tip_amount"])
        + float(row["tolls_amount"])
        + float(row["improvement_surcharge"]),
        2,
    )
    return abs(total_amount - total_amount_calculated) < shared.TOTAL_AMOUNT_CALCULATED_TOLERANCE / 2

In [7]:
def datetime_to_time_of_day(x: str) -> int:
    x: datetime = parse_datetime(x)
    x: time = x.time()
    return x.hour * 3600 + x.minute * 60 + x.second


def seconds_to_time(x: int) -> time:
    hour = int(x // 3600)
    minute = int((x % 3600) // 60)
    second = int(x % 60)
    return time(hour=hour, minute=minute, second=second)


def parse_vendor(x: str) -> int:
    return int(x)


def parse_pickup_datetime(x: str) -> datetime:
    return parse_datetime(x)


def parse_dropoff_datetime(x: str) -> datetime:
    return parse_datetime(x)


def parse_passenger_count(x: str) -> int:
    return int(x)


def parse_trip_distance(x: str) -> float:
    return float(x)


def parse_pickup_longitude(x: str) -> float | None:
    return float(x) if x != "0" else None


def parse_pickup_latitude(x: str) -> float | None:
    return float(x) if x != "0" else None


def parse_rate_code(x: str) -> int:
    return int(x)


def parse_store_and_fwd_flag(x: str) -> bool:
    return x == "Y"


def parse_dropoff_longitude(x: str) -> float | None:
    return float(x) if x != "0" else None


def parse_dropoff_latitude(x: str) -> float | None:
    return float(x) if x != "0" else None


def parse_payment_type(x: str) -> int:
    return int(x)


def parse_fare_amount(x: str) -> float:
    return float(x)


def parse_extra(x: str) -> float:
    return float(x)


def parse_mta_tax(x: str) -> float:
    return float(x)


def parse_tip_amount(x: str) -> float:
    return float(x)


def parse_tolls_amount(x: str) -> float:
    return float(x)


def parse_improvement_surcharge(x: str) -> float:
    return float(x)


def parse_total_amount(x: str) -> float:
    return float(x)

## 2. Exploration

### VendorID

In [8]:
if debug_exploration:
    vendors = rdd_nyc.map(lambda x: x["VendorID"]).persist(explore_storage_level)
    vendors_value_counts = vendors.countByValue()
    vendors_valid_counts = vendors.map(is_vendor_valid).countByValue()
    vendors.unpersist()

    print("Valid", dict(vendors_valid_counts))
    print("Values", dict(sorted(vendors_value_counts.items())))

Valid {True: 47248845}
Values {'1': 22227251, '2': 25021594}


### Pickup Time

In [9]:
if debug_exploration:
    pickup_times = rdd_nyc.map(lambda x: x["tpep_pickup_datetime"]).persist(explore_storage_level)
    pickup_times_valid_counts = pickup_times.map(is_pickup_time_valid).countByValue()
    pickup_times_time_of_day = pickup_times.map(datetime_to_time_of_day).map(lambda x: x // 3600).countByValue()
    pickup_times_month = pickup_times.map(parse_datetime).map(lambda x: x.month).countByValue()
    pickup_times.unpersist()

    print("Valid", dict(pickup_times_valid_counts))

Valid {True: 47248845}


### Dropoff Time

In [10]:
if debug_exploration:
    dropoff_times = rdd_nyc.map(lambda x: x["tpep_dropoff_datetime"]).persist(explore_storage_level)
    dropoff_times_valid_counts = dropoff_times.map(is_dropoff_time_valid).countByValue()
    dropoff_times_time_of_day = dropoff_times.map(datetime_to_time_of_day).map(lambda x: x // 3600).countByValue()
    dropoff_times_month = dropoff_times.map(parse_datetime).map(lambda x: x.month).countByValue()
    dropoff_times.unpersist()

    print("Valid", dict(dropoff_times_valid_counts))

Valid {True: 47248845}


### Passenger Count

In [11]:
if debug_exploration:
    passengers = rdd_nyc.map(lambda x: x["passenger_count"]).persist(explore_storage_level)
    passengers_valid_counts = passengers.map(is_passenger_count_valid).countByValue()
    passengers_value_counts = dict(sorted(passengers.countByValue().items()))
    passengers.unpersist()

    print("Valid", dict(dropoff_times_valid_counts))
    print("Values", passengers_value_counts)

Valid {True: 47248845}
Values {'0': 8214, '1': 33537914, '2': 6719430, '3': 1912291, '4': 911351, '5': 2551660, '6': 1607758, '7': 81, '8': 78, '9': 68}


### Trip Distance

In [12]:
if debug_exploration:
    trip_distances = rdd_nyc.map(lambda x: x["trip_distance"]).persist(explore_storage_level)
    trip_distances_valid_counts = trip_distances.map(is_trip_distance_valid).countByValue()
    trip_distances_valid = trip_distances.filter(is_trip_distance_valid).map(parse_trip_distance)
    trip_dinstances_ranges = trip_distances_valid.map(int).countByValue()
    trip_distances.unpersist()

    print("Valid", dict(dropoff_times_valid_counts))

Valid {True: 47248845}


### Pickup Longitude

In [13]:
if debug_exploration:
    pickup_longitudes = rdd_nyc.map(lambda x: x["pickup_longitude"]).persist(explore_storage_level)
    pickup_longitudes_valid_counts = pickup_longitudes.map(is_pickup_longitude_valid).countByValue()
    pickup_longitudes_range_counts = pickup_longitudes.map(float).map(round).map("~{}".format).countByValue()
    pickup_longitudes.unpersist()

    print("Valid", dict(pickup_longitudes_valid_counts))
    print("Ranges", dict(sorted(pickup_longitudes_range_counts.items())))

Valid {True: 47247751, False: 1094}
Ranges {'~-1': 2, '~-101': 2, '~-103': 1, '~-106': 1, '~-108': 1, '~-115': 1, '~-119': 1, '~-122': 105, '~-131': 1, '~-132': 1, '~-162': 1, '~-26': 3, '~-35': 1, '~-47': 1, '~-48': 1, '~-49': 2, '~-51': 2, '~-52': 2, '~-54': 1, '~-55': 2, '~-56': 3, '~-57': 5, '~-58': 1, '~-59': 2, '~-60': 2, '~-61': 15, '~-62': 5, '~-63': 4, '~-64': 9, '~-65': 13, '~-66': 61, '~-67': 12, '~-68': 24, '~-69': 50, '~-7': 1, '~-70': 57, '~-71': 65, '~-72': 96, '~-73': 905, '~-74': 46474089, '~-75': 369, '~-76': 69, '~-77': 158, '~-78': 40, '~-79': 31, '~-8': 2, '~-80': 42, '~-81': 11, '~-82': 9, '~-83': 9, '~-84': 3, '~-85': 7, '~-86': 1, '~-87': 2, '~-88': 3, '~-90': 2, '~-91': 1, '~-92': 1, '~0': 772527, '~39': 4, '~60': 1, '~79': 1, '~95': 1}


### Pickup Latitude

In [14]:
if debug_exploration:
    pickup_latitudes = rdd_nyc.map(lambda x: x["pickup_latitude"]).persist(explore_storage_level)
    pickup_latitudes_valid_counts = pickup_latitudes.map(is_pickup_latitude_valid).countByValue()
    pickup_latitudes_range_counts = pickup_latitudes.map(float).map(round).map("~{}".format).countByValue()
    pickup_latitudes.unpersist()

    print("Valid", dict(pickup_latitudes_valid_counts))
    print("Ranges", dict(sorted(pickup_latitudes_range_counts.items())))

Valid {True: 47247854, False: 991}
Ranges {'~-19': 1, '~-51': 1, '~-77': 4, '~0': 772527, '~10': 1, '~14': 2, '~15': 1, '~17': 2, '~19': 2, '~2': 1, '~20': 2, '~21': 1, '~23': 2, '~24': 1, '~25': 2, '~27': 3, '~28': 3, '~29': 6, '~30': 11, '~31': 10, '~32': 10, '~33': 14, '~34': 51, '~35': 22, '~36': 50, '~37': 155, '~38': 116, '~39': 193, '~40': 844, '~405': 1, '~41': 46474467, '~42': 95, '~43': 87, '~44': 63, '~45': 10, '~46': 14, '~47': 13, '~48': 9, '~49': 8, '~5': 1, '~50': 6, '~51': 7, '~52': 1, '~53': 4, '~54': 3, '~55': 1, '~56': 2, '~57': 4, '~59': 4, '~60': 1, '~61': 1, '~63': 1, '~67': 1, '~7': 1, '~8': 1, '~82': 1}


### Rate Code

In [15]:
if debug_exploration:
    rate_codes = rdd_nyc.map(lambda x: x["RateCodeID"]).persist(explore_storage_level)
    rate_codes_valid_counts = rate_codes.map(is_rate_code_valid).countByValue()
    rate_codes_value_counts = rate_codes.countByValue()
    rate_codes.unpersist()

    print("Valid", dict(rate_codes_valid_counts))
    print("Values", dict(sorted(rate_codes_value_counts.items())))

Valid {True: 47247503, False: 1342}
Values {'1': 46091898, '2': 922318, '3': 73869, '4': 18587, '5': 140377, '6': 454, '99': 1342}


### Store and Forward Flag

In [16]:
if debug_exploration:
    store_and_forwards = rdd_nyc.map(lambda x: x["store_and_fwd_flag"]).persist(explore_storage_level)
    store_and_forwards_valid_counts = store_and_forwards.map(is_store_and_forward_valid).countByValue()
    store_and_forwards_value_counts = store_and_forwards.countByValue()
    store_and_forwards.unpersist()

    print("Valid", dict(store_and_forwards_valid_counts))
    print("Values", dict(sorted(store_and_forwards_value_counts.items())))

Valid {True: 47248845}
Values {'N': 46938162, 'Y': 310683}


### Pickup Longitude

In [17]:
if debug_exploration:
    dropoff_longitudes = rdd_nyc.map(lambda x: x["dropoff_longitude"]).persist(explore_storage_level)
    dropoff_longitudes_valid_counts = dropoff_longitudes.map(is_dropoff_longitude_valid).countByValue()
    dropoff_longitudes_range_counts = dropoff_longitudes.map(float).map(round).map("~{}".format).countByValue()
    dropoff_longitudes.unpersist()

    print("Valid", dict(dropoff_longitudes_valid_counts))
    print("Ranges", dict(sorted(dropoff_longitudes_range_counts.items())))

Valid {True: 47247697, False: 1148}
Ranges {'~-1': 2, '~-100': 1, '~-101': 1, '~-103': 1, '~-12': 1, '~-122': 103, '~-123': 1, '~-152': 1, '~-162': 1, '~-26': 2, '~-33': 1, '~-35': 1, '~-4': 3, '~-47': 1, '~-48': 1, '~-49': 1, '~-52': 1, '~-55': 1, '~-56': 2, '~-57': 3, '~-58': 1, '~-59': 1, '~-60': 1, '~-61': 14, '~-62': 7, '~-63': 2, '~-64': 9, '~-65': 9, '~-66': 55, '~-67': 13, '~-68': 16, '~-69': 45, '~-70': 41, '~-706': 1, '~-71': 58, '~-72': 111, '~-73': 3185, '~-736': 2, '~-739': 1, '~-74': 46510069, '~-740': 1, '~-75': 846, '~-76': 60, '~-77': 160, '~-78': 37, '~-79': 32, '~-80': 44, '~-81': 10, '~-82': 10, '~-83': 7, '~-84': 3, '~-85': 5, '~-86': 1, '~-87': 2, '~-88': 1, '~-89': 1, '~-90': 1, '~-93': 2, '~-94': 2, '~-97': 2, '~0': 733841, '~39': 4, '~79': 2, '~85': 1}


### Pickup Latitude

In [18]:
if debug_exploration:
    dropoff_latitudes = rdd_nyc.map(lambda x: x["pickup_latitude"]).persist(explore_storage_level)
    dropoff_latitudes_valid_counts = dropoff_latitudes.map(is_dropoff_latitude_valid).countByValue()
    dropoff_latitudes_range_counts = dropoff_latitudes.map(float).map(round).map("~{}".format).countByValue()
    dropoff_latitudes.unpersist()

    print("Valid", dict(dropoff_latitudes_valid_counts))
    print("Ranges", dict(sorted(dropoff_latitudes_range_counts.items())))

Valid {True: 47247854, False: 991}
Ranges {'~-19': 1, '~-51': 1, '~-77': 4, '~0': 772527, '~10': 1, '~14': 2, '~15': 1, '~17': 2, '~19': 2, '~2': 1, '~20': 2, '~21': 1, '~23': 2, '~24': 1, '~25': 2, '~27': 3, '~28': 3, '~29': 6, '~30': 11, '~31': 10, '~32': 10, '~33': 14, '~34': 51, '~35': 22, '~36': 50, '~37': 155, '~38': 116, '~39': 193, '~40': 844, '~405': 1, '~41': 46474467, '~42': 95, '~43': 87, '~44': 63, '~45': 10, '~46': 14, '~47': 13, '~48': 9, '~49': 8, '~5': 1, '~50': 6, '~51': 7, '~52': 1, '~53': 4, '~54': 3, '~55': 1, '~56': 2, '~57': 4, '~59': 4, '~60': 1, '~61': 1, '~63': 1, '~67': 1, '~7': 1, '~8': 1, '~82': 1}


### Payment Type

In [19]:
if debug_exploration:
    payment_types = rdd_nyc.map(lambda x: x["payment_type"]).persist(explore_storage_level)
    payment_types_valid_counts = payment_types.map(is_payment_type_valid).countByValue()
    payment_types_value_counts = payment_types.countByValue()
    payment_types.unpersist()

    print("Valid", dict(payment_types_valid_counts))
    print("Values", dict(sorted(payment_types_value_counts.items())))

Valid {True: 47248845}
Values {'1': 30870614, '2': 16158086, '3': 164138, '4': 56004, '5': 3}


### Fare Amount

In [20]:
if debug_exploration:
    fare_amounts = rdd_nyc.map(lambda x: x["fare_amount"]).persist(explore_storage_level)
    fare_amounts_valid_counts = fare_amounts.map(is_fare_amount_valid).countByValue()
    fare_amounts.unpersist()

    print("Valid", dict(fare_amounts_valid_counts))

Valid {True: 47231632, False: 17213}


### Extra

In [21]:
if debug_exploration:
    extras = rdd_nyc.map(lambda x: x["extra"]).persist(explore_storage_level)
    extras_valid_counts = extras.map(is_extra_valid).countByValue()
    extras.unpersist()

    print("Valid", dict(extras_valid_counts))

Valid {True: 47240613, False: 8232}


### MTA Tax

In [22]:
if debug_exploration:
    mta_taxes = rdd_nyc.map(lambda x: x["mta_tax"]).persist(explore_storage_level)
    mta_taxes_valid_counts = mta_taxes.map(is_mta_tax_valid).countByValue()
    mta_taxes.unpersist()

    print("Valid", dict(mta_taxes_valid_counts))

Valid {True: 47232243, False: 16602}


### Tip Amount

In [23]:
if debug_exploration:
    tip_amounts = rdd_nyc.map(lambda x: x["tip_amount"]).persist(explore_storage_level)
    tip_amounts_valid_counts = tip_amounts.map(is_tip_amount_valid).countByValue()
    tip_amounts.unpersist()

    print("Valid", dict(tip_amounts_valid_counts))

Valid {True: 47248424, False: 421}


### Tolls Amount

In [24]:
if debug_exploration:
    tolls_amounts = rdd_nyc.map(lambda x: x["tolls_amount"]).persist(explore_storage_level)
    tolls_amounts_valid_counts = tolls_amounts.map(is_tolls_amount_valid).countByValue()
    tolls_amounts.unpersist()

    print("Valid", dict(tolls_amounts_valid_counts))

Valid {True: 47248734, False: 111}


### Improvement Surcharge

In [25]:
if debug_exploration:
    improvement_surcharges = rdd_nyc.map(lambda x: x["improvement_surcharge"]).persist(explore_storage_level)
    improvement_surcharges_valid_counts = improvement_surcharges.map(is_improvement_surcharge_valid).countByValue()
    improvement_surcharges.unpersist()

    print("Valid", dict(improvement_surcharges_valid_counts))

Valid {True: 47235773, False: 13072}


### Total Amount

In [26]:
if debug_exploration:
    total_amounts = rdd_nyc.map(lambda x: x["total_amount"]).persist(explore_storage_level)
    total_amounts_valid_counts = total_amounts.map(is_total_amount_valid).countByValue()
    total_amounts.unpersist()

    print("Valid", dict(total_amounts_valid_counts))

Valid {True: 47231583, False: 17262}


## 3. Filtering

In [27]:
validators_column: list[tuple[str, Callable[[str], bool]]] = [
    ("VendorID", is_vendor_valid),
    ("tpep_pickup_datetime", is_pickup_time_valid),
    ("tpep_dropoff_datetime", is_dropoff_time_valid),
    ("passenger_count", is_passenger_count_valid),
    ("trip_distance", is_trip_distance_valid),
    ("pickup_longitude", is_pickup_longitude_valid),
    ("pickup_latitude", is_pickup_latitude_valid),
    ("RateCodeID", is_rate_code_valid),
    ("store_and_fwd_flag", is_store_and_forward_valid),
    ("dropoff_longitude", is_dropoff_longitude_valid),
    ("dropoff_latitude", is_dropoff_latitude_valid),
    ("payment_type", is_payment_type_valid),
    ("fare_amount", is_fare_amount_valid),
    ("extra", is_extra_valid),
    ("mta_tax", is_mta_tax_valid),
    ("tip_amount", is_tip_amount_valid),
    ("tolls_amount", is_tolls_amount_valid),
    ("improvement_surcharge", is_improvement_surcharge_valid),
    ("total_amount", is_total_amount_valid),
]
validators_whole: list[tuple[str, Callable[[t.Row], bool]]] = [
    ("duration", is_duration_valid),
    ("total_calculated", is_total_amount_sum_valid),
]


def is_row_valid(row: t.Row) -> bool:
    for col, fil in validators_column:
        if not fil(row[col]):
            return False
    for _, fil in validators_whole:
        if not fil(row):
            return False
    return True


rdd_nyc_filtered = rdd_nyc.filter(lambda x: is_row_valid(x))

### Dropped Rows

In [28]:
from schemas import schema_nyc_dropped_silver


def add_reason(row: t.Row) -> t.Row:
    failed_column = next((col for col, fil in validators_column if not fil(row[col])), None)
    failed_whole = failed_column or next((col for col, fil in validators_whole if not fil(row)), None)
    return t.Row(reason=failed_column or failed_whole, **row.asDict())


if True:
    rdd_nyc_dropped = rdd_nyc.filter(lambda x: not is_row_valid(x)).map(add_reason)

    rdd_nyc_count = rdd_nyc.count()
    rdd_nyc_dropped_count = rdd_nyc_dropped.count()
    print(f"{rdd_nyc_dropped_count} dropped ({rdd_nyc_dropped_count / rdd_nyc_count:.2%})")
    if rdd_nyc_dropped_count > 0:
        rdd_nyc_dropped.toDF(schema=schema_nyc_dropped_silver).show()

22153 dropped (0.05%)


+----------------+--------+--------------------+---------------------+---------------+-------------+-------------------+------------------+----------+------------------+-------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|          reason|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|   pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag|  dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+----------------+--------+--------------------+---------------------+---------------+-------------+-------------------+------------------+----------+------------------+-------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|     fare_amount|       2| 2015-01-17 22:40:27|  2015-01-17 22:43:04|         

## 4 Transformations

**Cast Types**

In [29]:
type_casters: list[tuple[str, Callable[[str], Any]]] = [
    ("VendorID", parse_vendor),
    ("tpep_pickup_datetime", parse_pickup_datetime),
    ("tpep_dropoff_datetime", parse_dropoff_datetime),
    ("passenger_count", parse_passenger_count),
    ("trip_distance", parse_trip_distance),
    ("pickup_longitude", parse_pickup_longitude),
    ("pickup_latitude", parse_pickup_latitude),
    ("RateCodeID", parse_rate_code),
    ("store_and_fwd_flag", parse_store_and_fwd_flag),
    ("dropoff_longitude", parse_dropoff_longitude),
    ("dropoff_latitude", parse_dropoff_latitude),
    ("payment_type", parse_payment_type),
    ("fare_amount", parse_fare_amount),
    ("extra", parse_extra),
    ("mta_tax", parse_mta_tax),
    ("tip_amount", parse_tip_amount),
    ("tolls_amount", parse_tolls_amount),
    ("improvement_surcharge", parse_improvement_surcharge),
    ("total_amount", parse_total_amount),
]


def cast_types(row: t.Row) -> t.Row:
    d = row.asDict()

    for col, type_caster in type_casters:
        if isinstance(row[col], str):
            d[col] = type_caster(row[col])
    return t.Row(**d)


rdd_nyc_transformed = rdd_nyc_filtered.map(cast_types)

## 4. Saving results

In [30]:
from schemas import schema_nyc_silver
from shared import PATH_SILVER_NYC

df_nyc_silver = rdd_nyc_transformed.toDF(schema=schema_nyc_silver)
df_nyc_silver.write.parquet(PATH_SILVER_NYC, mode="overwrite")

In [31]:
spark.stop()